In [1]:
!pip install tensorflow

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [3]:
train_data_gen= ImageDataGenerator(rescale=1./255)
validation_data_gen =ImageDataGenerator(rescale=1./255)

#processing all train images
train_generator= train_data_gen.flow_from_directory('FER-2013/train', target_size=(48,48), batch_size=64, color_mode="grayscale", class_mode='categorical')

#process all test images
validation_generator = validation_data_gen.flow_from_directory('FER-2013/test', target_size=(48,48), batch_size=64, color_mode="grayscale", class_mode='categorical')


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
#creating model structure
model= Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))




In [5]:
learning_rate = 0.001  ############ original 0.0001
momentum = 0.9
nesterov = True
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=nesterov)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 22, 22, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 dropout (Dropout)           (None, 10, 10, 64)        0

In [52]:
#train the neural network/model 

model_info=model.fit_generator(
train_generator, steps_per_epoch= 28709//64, 
    ############ original epochs= 50,
    epochs=15,  
    validation_data=validation_generator,
    validation_steps=7178//64
)



C:\Users\Hp\AppData\Local\Temp\ipykernel_2684\2963240024.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_info=model.fit_generator(


Epoch 1/15
448/448 [==============================] - 277s 611ms/step - loss: 0.4982 - accuracy: 0.2068 - val_loss: 0.4052 - val_accuracy: 0.2472
Epoch 2/15
448/448 [==============================] - 170s 379ms/step - loss: 0.4000 - accuracy: 0.2297 - val_loss: 0.4055 - val_accuracy: 0.2472
Epoch 3/15
448/448 [==============================] - 180s 401ms/step - loss: 0.3977 - accuracy: 0.2363 - val_loss: 0.4047 - val_accuracy: 0.2469
Epoch 4/15
448/448 [==============================] - 1548s 3s/step - loss: 0.3962 - accuracy: 0.2362 - val_loss: 0.4044 - val_accuracy: 0.2471
Epoch 5/15
448/448 [==============================] - 163s 364ms/step - loss: 0.3955 - accuracy: 0.2374 - val_loss: 0.4046 - val_accuracy: 0.2469
Epoch 6/15
448/448 [==============================] - 161s 359ms/step - loss: 0.3946 - accuracy: 0.2408 - val_loss: 0.4044 - val_accuracy: 0.2471
Epoch 7/15
448/448 [==============================] - 149s 331ms/step - loss: 0.3940 - accuracy: 0.2425 - val_loss: 0.4038 - v

In [23]:

# saving model structure in json file
from keras.models import model_from_json
model_json=model.to_json()
with open("model.json", "w") as json_file:
    json_file.write('model')
model.save_weights('model.h5')

In [6]:
!pip install opencv-python

In [7]:
import cv2
import numpy as np
from keras.models import model_from_json

In [8]:
emotion_dict={  0:"Angry", 1:"Disgusted", 2:"Fearful", 3:"Happy", 4:"Neutral", 5:"Sad", 6:"Surprised",}
    

## - From this cell onwards, it will through error if the webcam is not connected. So to run these cells. Give related permission or connect external webcam.
## - Next cell onwards could take time to execute depending upon computational power.

In [ ]:
feed=cv2.VideoCapture(0)
#feed=cv2.VideoCapture("C:\\Users\\Hp\\OneDrive\\Desktop\\CSM 422 AI\\FER Project\\happy2.mp4")
while True:
    ret, frame =feed.read()
    frame=cv2.resize(frame, (1280, 720),cv2.INTER_AREA)
    if not ret:
        break
    
    face_detector=cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_default.xml")
    gray_frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)




In [ ]:
#detecting faces on videofeed
faces=face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)


In [ ]:
#preprocessing each face from camera and processing it
for (x,y,w,h) in faces:
    cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0,255, 0), 4)
    roi_gray_frame=gray_frame[y:y+h, x:x+w]
    cropped_img=np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48,48)), -1), 0)

In [ ]:
#predicting model
emotion_prediction= model.predict(cropped_img)
maxindex=int(np.argmax(emotion_prediction))
cv2.putText(frame,emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, CV2.LINE_AA)

In [ ]:
cv2.imshow('Emotion Detection', frame)
if cv2.waitKey(1) & 0xFF ==ord('q'):
    cap.release()
cv2.destroyAllWindows()
